## Importing libraries

In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers, models

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset


Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
 99% 2.02G/2.04G [00:30<00:00, 153MB/s]
100% 2.04G/2.04G [00:30<00:00, 72.6MB/s]


In [ ]:
!ls

plantvillage-dataset.zip  sample_data


## Unzip the downloaded dataset

In [ ]:

with ZipFile("plantvillage-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

## Set the path to dataset

In [ ]:
data_dir = "/content/plantvillage dataset/color"

## Define image size and batch size

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

## ImageDataGenerator for data loading and augmentation

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

## Training data generator

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 43456 images belonging to 38 classes.


## Validation data generator

In [ ]:
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 10849 images belonging to 38 classes.


## Loading pre-trained MobileNetV2 model


In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the base model
base_model.trainable = False

**GlobalAveragePooling2D():** This layer reduces the spatial dimensions to a single value per feature map, effectively converting the 2D feature maps to 1D vectors.

**Dense(128, activation='relu'):** Adds a fully connected layer with 128 units and ReLU activation.

**Dropout(0.5):** Dropout layer for regularization, randomly deactivating 50% of neurons during training to prevent overfitting.

**Dense(train_generator.num_classes, activation='softmax'):** Final dense layer, where train_generator.num_classes is the number of classes in your dataset.

In [ ]:
# Add custom layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=3,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


In [ ]:
print("Evaluating model...")
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
1247/1247 ━━━━━━━━━━━━━━━━━━━━ 1576s 1s/step - accuracy: 0.9417 - loss: 0.1695
Validation Accuracy: 94.13%


In [ ]:
model.save('mymodel.h5')

In [ ]:
model.save('my_model.keras')